# Simulation: Patient Participation Tokenomics

All rights reserved, unlicensed.

Loads `config.json` and simulates tokens/tiers (0/3/6/9%). No PHI.

In [ ]:
import json, math, random
with open('config.json') as f:
    cfg = json.load(f)
tiers = cfg['tiers_percent']
thr = cfg['thresholds_tokens']
cap = cfg['cap_monthly_tokens_per_user']
tps = cfg['token_per_session']
pen = cfg['no_show_penalty_percent'] / 100.0
k   = cfg['decay_k']
months = cfg['simulation']['months']
cohorts = cfg['simulation']['cohorts']
pop = {name: int(1000 * cohorts[name]['size']) for name in cohorts}
print('Population by cohort:', pop)


In [ ]:
import math, random

def tier_for_tokens(tokens, thr, tiers):
    if tokens >= thr[3]: return tiers[3]
    if tokens >= thr[2]: return tiers[2]
    if tokens >= thr[1]: return tiers[1]
    return tiers[0]

def simulate(pop, cohorts, months, tps, cap, pen, k, thr, tiers, cfg):
    users = []
    for name, size in pop.items():
        ar = cohorts[name]['attendance_rate']
        nr = cohorts[name]['no_show_rate']
        for _ in range(size):
            users.append({'cohort':name,'tokens':0.0,'penalty':0.0,'inact':0,'ar':ar,'nr':nr})
    monthly=[]; total_sessions=0; total_cost=0.0
    for m in range(1, months+1):
        sess=0; cost=0.0
        for u in users:
            attends = (random.random() < u['ar']*math.exp(-k*u['inact']))
            if attends:
                u['inact']=0
                if random.random() < u['nr']:
                    u['penalty'] = min(1.0, u['penalty'] + pen)
                else:
                    sess += 1
                    add = tps * (1.0 - u['penalty'])
                    add = max(0.0, min(add, cap))
                    u['tokens'] += add
            else:
                u['inact'] += 1
            u['penalty'] *= 0.5
        for u in users:
            disk = tier_for_tokens(u['tokens'], thr, tiers)/100.0
            if random.random() < cohorts[u['cohort']]['attendance_rate']:
                cost += disk * cfg['discount_cost_per_session_usd']
        total_sessions += sess
        total_cost += cost
        issued = sum(u['tokens'] for u in users)
        monthly.append({'month':m,'issued_tokens':issued,'sessions':sess,'discount_cost_usd':cost})
    return monthly, {'issued_tokens':issued,'sessions':total_sessions,'discount_cost_usd':total_cost}

monthly, kpis = simulate(pop, cohorts, months, tps, cap, pen, k, thr, tiers, cfg)
print('Last month snapshot:', monthly[-1])
print('Aggregate KPIs:', kpis)
